### Import the dependencies, libraries, modules, and the weather_api_key

In [ ]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Add the requests library from Anaconda
# import requests
# Import the datetime module from the datetime library.
from datetime import datetime

In [ ]:
# Import the API key.
from config import weather_api_key

### Create a set of 2,000 random latitudes and longitudes

In [ ]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [ ]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

### Get the nearest city using the citypy module

In [ ]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [ ]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

### Perform an API call with the OpenWeatherMap

In [ ]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

### Retrieve the following from the API call:  Latitude and longitude, Maximum temperature, Percent humidity, Percent cloudiness, Wind speed, Weather description

In [ ]:
import requests

# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
     # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_weather_description = city_weather["main"]["description"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                         "Current Description": city_weather_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    

In [ ]:
len(city_data)

###  Add the retrieved data to a new DataFrame

In [11]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Atuona,-9.8000,-139.0333,81.05,77,81,10.58,PF,2021-04-06 22:56:17
1,Tommot,58.9564,126.2925,-0.62,89,26,2.39,RU,2021-04-06 22:59:34
2,Kosonsoy,41.2561,71.5508,48.20,61,0,6.91,UZ,2021-04-06 22:59:34
3,Henties Bay,-22.1160,14.2845,63.18,83,0,7.81,NA,2021-04-06 22:59:35
4,Airai,-8.9266,125.4092,64.74,79,77,0.56,TL,2021-04-06 22:47:21
5,Saint-Philippe,-21.3585,55.7679,77.00,69,0,4.61,RE,2021-04-06 22:59:35
6,Ostersund,63.1792,14.6357,33.01,69,75,17.27,SE,2021-04-06 22:59:35
7,Bluff,-46.6000,168.3333,57.99,82,100,8.99,NZ,2021-04-06 22:56:09
8,Gat,31.6100,34.7642,69.01,37,85,2.17,IL,2021-04-06 22:50:01
9,Geraldton,-28.7667,114.6000,68.00,77,60,5.75,AU,2021-04-06 22:59:36


### Adjust the column order

In [12]:
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Atuona,PF,2021-04-06 22:56:17,-9.8000,-139.0333,81.05,77,81,10.58
1,Tommot,RU,2021-04-06 22:59:34,58.9564,126.2925,-0.62,89,26,2.39
2,Kosonsoy,UZ,2021-04-06 22:59:34,41.2561,71.5508,48.20,61,0,6.91
3,Henties Bay,NA,2021-04-06 22:59:35,-22.1160,14.2845,63.18,83,0,7.81
4,Airai,TL,2021-04-06 22:47:21,-8.9266,125.4092,64.74,79,77,0.56
5,Saint-Philippe,RE,2021-04-06 22:59:35,-21.3585,55.7679,77.00,69,0,4.61
6,Ostersund,SE,2021-04-06 22:59:35,63.1792,14.6357,33.01,69,75,17.27
7,Bluff,NZ,2021-04-06 22:56:09,-46.6000,168.3333,57.99,82,100,8.99
8,Gat,IL,2021-04-06 22:50:01,31.6100,34.7642,69.01,37,85,2.17
9,Geraldton,AU,2021-04-06 22:59:36,-28.7667,114.6000,68.00,77,60,5.75


### Export the DataFrame as a CSV file

In [13]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")